In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr,spearmanr
import scipy
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score 
from sklearn import linear_model
from scipy import stats
import statistics
import os
from sklearn.preprocessing import LabelEncoder
from numpy import percentile

In [ ]:
dataset_dir = './data'

In [ ]:
df_hosp = pd.read_csv(os.path.join(dataset_dir, 'hosp_withoutcbu_auck.csv'))
data_hen = pd.read_csv(os.path.join(dataset_dir, 'air_quality_hen.csv'))

In [ ]:
#df_hosp = pd.read_csv("H:/asthma/data/hosp_withoutcbu_auck.csv")
#data_hen = pd.read_csv("H:/asthma/data/air_quality_hen.csv")

In [ ]:
df_hosp

In [ ]:
data_hen

In [ ]:
df_bc = pd.DataFrame(data_hen[['Date', 'BC(370)', 'BC(880)']])

In [ ]:
df_bc

In [ ]:
df_bc = pd.DataFrame(df_bc.loc[(df_bc['Date'] >= '2018-01-01') & (df_bc['Date'] <= '2021-01-01')])

In [ ]:
print("Mean of the 370 is % s ", round(np.nanmean(df_bc['BC(370)']), 2))

In [ ]:
print("Stddev of the 370 is % s ", round(np.nanstd(df_bc['BC(370)']), 2))

In [ ]:
print("Stddev of the 370 is % s ", np.nanvar(df_bc['BC(370)']))

In [ ]:
sns.displot(df_bc, x="BC(370)" ,kind= "kde")

In [ ]:
sns.boxplot(df_bc['BC(370)'])

In [ ]:
df_bc_per = df_bc[df_bc['BC(370)'].notnull()]

In [ ]:
q25, q75 = percentile(df_bc_per['BC(370)'], 25), percentile(df_bc_per['BC(370)'], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in df_bc['BC(370)'] if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in df_bc['BC(370)'] if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

In [ ]:
df_bc

In [ ]:
df_bc['capped_bc(370)'] = np.where(
    df_bc['BC(370)'] > upper,
    upper,
    np.where(
        df_bc['BC(370)'] < lower,
        lower,
        df_bc['BC(370)']
    )
)

In [ ]:
df_bc

In [ ]:
sns.boxplot(df_bc['capped_bc(370)'])

In [ ]:
df_bc

In [ ]:
#df_bc = pd.DataFrame(df_bc.loc[(df_bc['Date'] >= '2018-01-01') & (df_bc['Date'] <= '2021-01-01')])

In [ ]:
df_bc.isnull().sum()

In [ ]:
df_bc['capped_bc(370)'] = df_bc['capped_bc(370)'].fillna(df_bc['capped_bc(370)'].median())

In [ ]:
print("Mean of the 370 is % s ", round(np.nanmean(df_bc['capped_bc(370)']), 2))

In [ ]:
print("Stddev of the 370 is % s ", round(np.nanstd(df_bc['capped_bc(370)']), 2))

In [ ]:
print("Mean of the 880 is % s ", round(np.nanmean(df_bc['BC(880)']), 2))

In [ ]:
print("Stddev of the 880 is % s ", round(np.nanstd(df_bc['BC(880)']), 2))

In [ ]:
print("Stddev of the 880 is % s ", np.nanvar(df_bc['BC(880)']))

In [ ]:
sns.displot(df_bc, x="BC(880)" ,kind= "kde")

In [ ]:
sns.boxplot(df_bc['BC(880)'])

In [ ]:
df_bc_per88 = df_bc[df_bc['BC(880)'].notnull()]

In [ ]:
q25, q75 = percentile(df_bc_per88['BC(880)'], 25), percentile(df_bc_per88['BC(880)'], 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in df_bc['BC(880)'] if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in df_bc['BC(880)'] if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

In [ ]:
df_bc['capped_bc(880)'] = np.where(
    df_bc['BC(880)'] > upper,
    upper,
    np.where(
        df_bc['BC(880)'] < lower,
        lower,
        df_bc['BC(880)']
    )
)

In [ ]:
df_bc

In [ ]:
sns.boxplot(df_bc['capped_bc(880)'])

In [ ]:
df_bc.isnull().sum()

In [ ]:
df_hosp.rename(columns = {'Admit Date' : 'Date'}, inplace = True)

In [ ]:
df_hosp = pd.DataFrame(df_hosp.loc[(df_hosp['Date'] >= '2018-01-01') & (df_hosp['Date'] <= '2021-01-01')])

In [ ]:
left = df_bc
right = df_hosp
#left['Date(NZST)'] = pd.to_datetime(left["Date(NZST)"])
#right['Date(NZST)'] = pd.to_datetime(right["Date(NZST)"])
df_bc = pd.merge(left, right, on=["Date"], how= "outer", validate="one_to_one").sort_values(by = "Date").reset_index(drop= True)

In [ ]:
#df_bc = df_bc[df_bc['capped_bc(880)'].notnull()]
df_bc['capped_bc(880)'] = df_bc['capped_bc(880)'].fillna(df_bc['capped_bc(880)'].median())

In [ ]:
print("Mean of the 880 is % s ", round(np.nanmean(df_bc['capped_bc(880)']), 2))

In [ ]:
print("Stddev of the 880 is % s ", round(np.nanstd(df_bc['capped_bc(880)']), 2))

In [ ]:
df_bc['shifted'] = df_bc['scaled data'].shift(0)

In [ ]:
#df_bc = df_bc[df_bc['BC(370)'].notnull()]

In [ ]:
df_bc

In [ ]:
df_bc = df_bc[df_bc['shifted'].notnull()]

In [ ]:
df_bc

In [ ]:
a = stats.zscore(df_bc['capped_bc(880)'])

In [ ]:
d = stats.zscore(df_bc['capped_bc(370)'])

In [ ]:
b = df_bc['status']

In [ ]:
c = df_bc['shifted']

In [ ]:
pearsonr(a, c)[0].round(decimals = 4)

In [ ]:
pearsonr(a, c)[1].round(decimals = 4)

In [ ]:
pearsonr(d, c)[0].round(decimals = 4)

In [ ]:
pearsonr(d, c)[1].round(decimals = 4)

In [ ]:
df_bc['capped_bc(370)'] = df_bc['capped_bc(370)'].shift(-1)
df_bc['capped_bc(880)'] = df_bc['capped_bc(880)'].shift(-1)

In [ ]:
df_bc = df_bc[df_bc['capped_bc(370)'].notnull()]

In [ ]:
print("bc370" + " " + str(pearsonr(df_bc['capped_bc(370)'],df_bc['scaled data'])[0].round(decimals = 4)))
print("bc880" + " " + str(pearsonr(df_bc['capped_bc(880)'],df_bc['scaled data'])[0].round(decimals = 4)))

In [ ]:
import seaborn as sns
fig = sns.regplot(x= df_bc["capped_bc(370)"], y= df_bc['Daily counts'])
fig.set(xlabel='BC(370)', ylabel='Hospital Data')
plt.show()

In [ ]:
import seaborn as sns
fig = sns.regplot(x= df_bc["capped_bc(880)"], y= df_bc['Daily counts'])
fig.set(xlabel='BC(880)', ylabel='Hospital Data')
plt.show()

In [ ]:
spearmanr(a, b)[0].round(decimals = 4)

In [ ]:
spearmanr(a, b)[1].round(decimals = 4)

In [ ]:
#plt.scatter(data_hosp['Admit Date'],  a)
#plt.scatter(data_hosp['Admit Date'],  c)
#plt.show()

In [ ]:
#df_hosp.rename(columns = {'Admit Date' : 'Date'}, inplace = True)

In [ ]:
#left = df_bc
#right = df_hosp
#left['Date(NZST)'] = pd.to_datetime(left["Date(NZST)"])
#right['Date(NZST)'] = pd.to_datetime(right["Date(NZST)"])
#df_bc = pd.merge(left, right, on=["Date"], how= "outer", validate="one_to_one").sort_values(by = "Date")

In [ ]:
df_bc

In [ ]:
df_bc.columns

In [ ]:
df_bc = pd.DataFrame(df_bc[['Date', 'capped_bc(370)', 'capped_bc(880)']])

In [ ]:
df_bc

In [ ]:
##df_bc_write.to_csv('H:/asthma/data/bc_testing.csv', index = False)

In [ ]:
#df_bc_write.to_csv((os.path.join(dataset_dir, 'bc_testing.csv')), index = False)